# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [281]:
import sys
print(sys.executable)

C:\Users\bened\miniconda3\envs\DSHF3\python.exe


In [282]:
# Szükséges könyvtárak importálása
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy import stats
from datetime import datetime
import glob

warnings.filterwarnings('ignore')

# Ábrázolási beállítások
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Könyvtárak importálva!")


Könyvtárak importálva!


In [283]:
# Könyvtárak elérési útjai
data_path = Path('data/usa_stocks_data')
constituents_path = data_path / 'index/constituents'
idxperf_path = data_path / 'index/perf'
stock_prices_path = data_path / 'stock_prices'

# Index részvényeinek betöltése
sp500_companies1 = pd.read_csv(constituents_path / 'sp500.csv', header=0, index_col=0)
sp500_companies2 = pd.read_csv(constituents_path / 'sp500_companies.csv', header=0, index_col=0)
dowjones_companies = pd.read_csv(constituents_path / 'dowjones_companies.csv', header=0, index_col=0)
nasdaq100_companies = pd.read_csv(constituents_path / 'nasdaq100_companies.csv', header=0, index_col=0)

# Index teljesítmény adatok betöltése
sp500_perf = pd.read_csv(idxperf_path / 'SP500_perf.csv', parse_dates=['Date'], header=0)
dowjones_perf = pd.read_csv(idxperf_path / 'Dow_Jones_perf.csv', parse_dates=['Date'], header=0)
nasdaq100_perf = pd.read_csv(idxperf_path / 'Nasdaq100_perf.csv', parse_dates=['Date'], header=0)

print("Indexek betöltve!")

Indexek betöltve!


## S&P 500

Amint látható, S&P 500 indexhez 2 fájl is található. Ezeket megvizsgálom, és a jelenlegi állapotához közelebbit használom a továbbiakban

In [284]:
# Szimbólumok összehasonlítása
print(f"sp500_companies.csv rekordjainak száma: {len(sp500_companies1)}")
print(f"sp500.csv rekordjainak száma: {len(sp500_companies2)}")

sp500_symbols = set(sp500_companies1['Symbol'].unique())
sp500_companies_symbols = set(sp500_companies2['Symbol'].unique())

print(f"\nSP500.csv egyedi szimbólumok száma: {len(sp500_symbols)}")
print(f"SP500_companies.csv egyedi szimbólumok száma: {len(sp500_companies_symbols)}")

# Szimbólumok, amelyek csak az egyikben vannak
only_sp500 = sp500_symbols - sp500_companies_symbols
only_sp500_companies = sp500_companies_symbols - sp500_symbols
common_symbols = sp500_symbols & sp500_companies_symbols

print(f"\nKözös szimbólumok: {len(common_symbols)}")
print(f"Csak SP500-ban lévő szimbólumokszáma: {len(only_sp500)}")
print(f"Csak SP500_companies-ben lévő szimbólumok száma: {len(only_sp500_companies)}")

print(f"\nCsak SP500-ban lévő szimbólumok: {list(only_sp500)}")
print(f"Csak SP500_companies-ben lévő szimbólumok: {list(only_sp500_companies)}")


sp500_companies.csv rekordjainak száma: 503
sp500.csv rekordjainak száma: 501

SP500.csv egyedi szimbólumok száma: 503
SP500_companies.csv egyedi szimbólumok száma: 501

Közös szimbólumok: 500
Csak SP500-ban lévő szimbólumokszáma: 3
Csak SP500_companies-ben lévő szimbólumok száma: 1

Csak SP500-ban lévő szimbólumok: ['ABNB', 'VLTO', 'BX']
Csak SP500_companies-ben lévő szimbólumok: ['NWL']


Azt, hogy nem 500 rekordot tartalmaznak a fájlok az magyarázza, hogy bizonyos cégeknek több osztályú részvénye is része a rekordoknak, pl.: GOOG (Class B), GOOGL (Class A)

In [285]:
#Attribútumok vizsgálata
print(f"\nsp500.csv oszlopok:\n {list(sp500_companies1.columns)}\n")
print(f"\nsp500_companies.csv oszlopok:\n {list(sp500_companies2.columns)}\n")


sp500.csv oszlopok:
 ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded', 'Shares Outstanding']


sp500_companies.csv oszlopok:
 ['Symbol', 'Company Name', 'Industry', 'Sector', 'Founded', 'shares outstanding']



A rekordok számának viszgálatából kiderült, hogy egyik fájlban sincs duplikált adat.

Az sp500_companies.csv-ben található egyedül az NWL (Newell Brands Inc.), ami 2023 szeptember 18.-án kikerült az indexből.
Az sp500.csv-ben található egyedül 'ABNB', 'VLTO', 'BX', (Airbnb, Inc., Blackstone Inc., Veralto Corp.), amik 2023 szeptember 18.-án, és 2023 október 2.-án kerültek fel az indexre.
Ezek alapján az sp500.csv aktuálisabb információt tartalmaz.

Emellet az sp500.csv több információt tartalmaz a cégekről, és összes sp500_companies.csv-ben találhtó attribútum is jelen van.

# Tehát innentől az sp500.csv-t használom az S&P 500 teljesítményének vizsgálatára.


In [286]:
sp500 = sp500_companies1

Adatok tisztítása

In [287]:
text_cols = ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location']
for col in text_cols:
    sp500[col] = sp500[col].astype('string').str.strip()

sp500['Date added'] = pd.to_datetime(sp500['Date added'], errors='coerce')

sp500['CIK'] = sp500['CIK'].astype(str).str.zfill(10)

import re
def extract_year(x):
    if pd.isna(x):
        return pd.NA
    years = [int(y) for y in re.findall(r'(\d{4})', str(x))]
    return min(years) if years else pd.NA

sp500['Founded'] = sp500['Founded'].apply(extract_year)
sp500['Founded'] = pd.to_numeric(sp500['Founded'], errors='coerce')



print(sp500.isna().sum())
print(sp500[sp500["Founded"].isna()].Symbol)
print(sp500[sp500["Date added"].isna()].Symbol)
sp500.head()


Symbol                    0
Security                  0
GICS Sector               0
GICS Sub-Industry         1
Headquarters Location     0
Date added               12
CIK                       0
Founded                   0
Shares Outstanding        0
dtype: int64
Series([], Name: Symbol, dtype: string)
156       D
186      ES
211     FCX
243     HUM
404     ROK
415     SRE
435    TROW
445     TXN
457     USB
482      WM
490      WY
491     WHR
Name: Symbol, dtype: string


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Shares Outstanding
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902,5.519920e+08
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916,1.504910e+08
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888,1.735360e+09
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,1888,1.765050e+09
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989,6.295080e+08


# S&P 500 Attribútumok vizsgálata

In [288]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, 0 to 502
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Symbol                 503 non-null    string        
 1   Security               503 non-null    string        
 2   GICS Sector            503 non-null    string        
 3   GICS Sub-Industry      502 non-null    string        
 4   Headquarters Location  503 non-null    string        
 5   Date added             491 non-null    datetime64[ns]
 6   CIK                    503 non-null    object        
 7   Founded                503 non-null    int64         
 8   Shares Outstanding     503 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), string(5)
memory usage: 39.3+ KB


Hiányzó adatok vizsgálata

In [289]:
print(sp500[sp500["GICS Sub-Industry"].isna()])

    Symbol Security  GICS Sector GICS Sub-Industry   Headquarters Location  \
468   VLTO  Veralto  Industrials              <NA>  Waltham, Massachusetts   

    Date added         CIK  Founded  Shares Outstanding  
468 2023-10-02  0001967680     2023         246291005.0  


Hiányzó adat: Environmental & Facilities Services, ennek hozzáadása

In [290]:
sp500.loc[sp500["Symbol"] == "VLTO", "GICS Sub-Industry"] = "Environmental & Facilities Services"

In [291]:
print(sp500[sp500["Date added"].isna()].Symbol)

156       D
186      ES
211     FCX
243     HUM
404     ROK
415     SRE
435    TROW
445     TXN
457     USB
482      WM
490      WY
491     WHR
Name: Symbol, dtype: string


In [292]:
missing_map = {
    "D":   "2016-11-30",
    "ES":  "2009-07-24",
    "FCX": "2011-07-01",
    "HUM": "2012-12-10",
    "ROK": "2000-03-12",
    "SRE": "2017-03-17",
    "TROW":"2019-07-29",
    "TXN": "2001-03-12",
    "USB": "1999-11-01",
    "WM":  "1998-08-31",
    "WY":  "1979-10-01",
    "WHR": "1959-04-01",
}

for sym, date in missing_map.items():
    sp500.loc[(sp500["Symbol"] == sym) & (sp500["Date added"].isna()), "Date added"] = date

In [293]:
sp500.info()
sp500.head()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, 0 to 502
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Symbol                 503 non-null    string        
 1   Security               503 non-null    string        
 2   GICS Sector            503 non-null    string        
 3   GICS Sub-Industry      503 non-null    string        
 4   Headquarters Location  503 non-null    string        
 5   Date added             503 non-null    datetime64[ns]
 6   CIK                    503 non-null    object        
 7   Founded                503 non-null    int64         
 8   Shares Outstanding     503 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), string(5)
memory usage: 39.3+ KB


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Shares Outstanding
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902,5.519920e+08
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916,1.504910e+08
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888,1.735360e+09
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,1888,1.765050e+09
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989,6.295080e+08


A hiányzó adatok ki lettek töltve.

In [294]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, 0 to 502
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Symbol                 503 non-null    string        
 1   Security               503 non-null    string        
 2   GICS Sector            503 non-null    string        
 3   GICS Sub-Industry      503 non-null    string        
 4   Headquarters Location  503 non-null    string        
 5   Date added             503 non-null    datetime64[ns]
 6   CIK                    503 non-null    object        
 7   Founded                503 non-null    int64         
 8   Shares Outstanding     503 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), string(5)
memory usage: 39.3+ KB


Az szektorok száma megegyezik a GICS által meghatározottnak, az iparágaké nem, mivel vannak olyanok amik nem elemei az indexnek.

In [295]:
sp500_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24060 entries, 0 to 24059
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          24060 non-null  object 
 1   Open          24060 non-null  float64
 2   High          24060 non-null  float64
 3   Low           24060 non-null  float64
 4   Close         24060 non-null  float64
 5   Volume        24060 non-null  int64  
 6   Dividends     24060 non-null  float64
 7   Stock Splits  24060 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.5+ MB


In [296]:
sp500_perf['Date'] = pd.to_datetime(sp500_perf['Date'], errors='coerce')
sp500_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24060 entries, 0 to 24059
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                    
---  ------        --------------  -----                    
 0   Date          10803 non-null  datetime64[ns, UTC-05:00]
 1   Open          24060 non-null  float64                  
 2   High          24060 non-null  float64                  
 3   Low           24060 non-null  float64                  
 4   Close         24060 non-null  float64                  
 5   Volume        24060 non-null  int64                    
 6   Dividends     24060 non-null  float64                  
 7   Stock Splits  24060 non-null  float64                  
dtypes: datetime64[ns, UTC-05:00](1), float64(6), int64(1)
memory usage: 1.5 MB


# Nasdaq 100


In [297]:
nasdaq100_companies.head()
nasdaq100_companies.info()
nasdaq100_companies

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Symbol              101 non-null    object 
 1   Company Name        101 non-null    object 
 2   Industry            101 non-null    object 
 3   Sector              101 non-null    object 
 4   Founded             101 non-null    object 
 5   shares outstanding  101 non-null    float64
dtypes: float64(1), object(5)
memory usage: 5.5+ KB


,Symbol,Company Name,Industry,Sector,Founded,shares outstanding
0,AAPL,Apple,Consumer Electronics,Technology,1977,1.563420e+10
1,MSFT,Microsoft,Software - Infrastructure,Technology,1975,7.429760e+09
2,GOOGL,Alphabet,Internet Content & Information,Communication Services,1998,1.265650e+10
3,GOOG,Alphabet,Internet Content & Information,Communication Services,1998,1.255370e+10
4,AMZN,Amazon.com,Internet Retail,Consumer Discretionary,1994,1.031780e+10
...,...,...,...,...,...,...
96,WBA,Walgreens Boots Alliance,Pharmaceutical Retailers,Healthcare,1901,8.632610e+08
97,ZM,Zoom Video Communications,Software - Application,Technology,2011,3.009570e+08
98,SIRI,Sirius XM Holdings,Entertainment,Communication Services,-,3.842400e+09
99,ENPH,Enphase Energy,Solar,Technology,2006,1.363550e+08


In [298]:
text_cols = ['Symbol', 'Company Name', 'Industry', 'Sector']
for col in text_cols:
    nasdaq100_companies[col] = nasdaq100_companies[col].astype('string').str.strip()

nasdaq100_companies['Founded'] = pd.to_numeric(nasdaq100_companies['Founded'], errors='coerce')


print(nasdaq100_companies.isna().sum())
print(nasdaq100_companies[nasdaq100_companies["Founded"].isna()].Symbol)



Symbol                 0
Company Name           0
Industry               0
Sector                 0
Founded               12
shares outstanding     0
dtype: int64
8     AVGO
24     HON
35    MDLZ
45     CSX
52    MNST
54      JD
66      ON
68    SGEN
69     EXC
75    IDXX
78     BKR
98    SIRI
Name: Symbol, dtype: string


In [299]:
missing_map = {
    "AVGO": 1961,
    "HON":  1885,
    "MDLZ": 1903,
    "CSX":  1980,
    "MNST": 1935,
    "JD":   1998,
    "ON":   1999,
    "SGEN": 1998,
    "EXC":  2000,
    "IDXX": 1983,
    "BKR":  1908,
    "SIRI": 2008
}

for sym, date in missing_map.items():
    nasdaq100_companies.loc[(nasdaq100_companies["Symbol"] == sym) & (nasdaq100_companies["Founded"].isna()), "Founded"] = date

nasdaq100_companies['Founded'] = nasdaq100_companies['Founded'].astype(int)

In [300]:
nasdaq100_companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Symbol              101 non-null    string 
 1   Company Name        101 non-null    string 
 2   Industry            101 non-null    string 
 3   Sector              101 non-null    string 
 4   Founded             101 non-null    int64  
 5   shares outstanding  101 non-null    float64
dtypes: float64(1), int64(1), string(4)
memory usage: 5.5 KB


In [305]:
nasdaq100_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9585 entries, 0 to 9584
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          9585 non-null   object 
 1   Open          9585 non-null   float64
 2   High          9585 non-null   float64
 3   Low           9585 non-null   float64
 4   Close         9585 non-null   float64
 5   Volume        9585 non-null   int64  
 6   Dividends     9585 non-null   float64
 7   Stock Splits  9585 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 599.2+ KB


In [306]:
nasdaq100_perf['Date'] = pd.to_datetime(nasdaq100_perf['Date'], errors='coerce')
nasdaq100_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9585 entries, 0 to 9584
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                    
---  ------        --------------  -----                    
 0   Date          5830 non-null   datetime64[ns, UTC-04:00]
 1   Open          9585 non-null   float64                  
 2   High          9585 non-null   float64                  
 3   Low           9585 non-null   float64                  
 4   Close         9585 non-null   float64                  
 5   Volume        9585 non-null   int64                    
 6   Dividends     9585 non-null   float64                  
 7   Stock Splits  9585 non-null   float64                  
dtypes: datetime64[ns, UTC-04:00](1), float64(6), int64(1)
memory usage: 599.2 KB


# Dow Jones

In [301]:
dowjones_companies.info()
dowjones_companies.columns

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        30 non-null     object
 1   Company Name  30 non-null     object
 2   Industry      30 non-null     object
 3   Sector        30 non-null     object
 4   Founded       30 non-null     object
dtypes: object(5)
memory usage: 1.4+ KB


Index(['Symbol', 'Company Name', 'Industry', 'Sector', 'Founded'], dtype='object')

In [302]:
text_cols = ['Symbol', 'Company Name', 'Industry', 'Sector']
for col in text_cols:
    dowjones_companies[col] = dowjones_companies[col].astype('string').str.strip()

dowjones_companies['Founded'] = pd.to_numeric(dowjones_companies['Founded'], errors='coerce')


print(dowjones_companies.isna().sum())
print(dowjones_companies[dowjones_companies["Founded"].isna()].Symbol)



Symbol          0
Company Name    0
Industry        0
Sector          0
Founded         2
dtype: int64
22    HON
28    DOW
Name: Symbol, dtype: string


In [303]:
missing_map = {
    "HON": 1906,
    "DOW":  1897
}

for sym, date in missing_map.items():
    dowjones_companies.loc[(dowjones_companies["Symbol"] == sym) & (dowjones_companies["Founded"].isna()), "Founded"] = date

dowjones_companies['Founded'] = dowjones_companies['Founded'].astype(int)

dowjones_companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        30 non-null     string
 1   Company Name  30 non-null     string
 2   Industry      30 non-null     string
 3   Sector        30 non-null     string
 4   Founded       30 non-null     int64 
dtypes: int64(1), string(4)
memory usage: 1.4 KB


In [307]:
dowjones_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8004 entries, 0 to 8003
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          8004 non-null   object 
 1   Open          8004 non-null   float64
 2   High          8004 non-null   float64
 3   Low           8004 non-null   float64
 4   Close         8004 non-null   float64
 5   Volume        8004 non-null   int64  
 6   Dividends     8004 non-null   float64
 7   Stock Splits  8004 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 500.4+ KB


In [308]:
dowjones_perf['Date'] = pd.to_datetime(dowjones_perf['Date'], errors='coerce')
dowjones_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8004 entries, 0 to 8003
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                    
---  ------        --------------  -----                    
 0   Date          3043 non-null   datetime64[ns, UTC-05:00]
 1   Open          8004 non-null   float64                  
 2   High          8004 non-null   float64                  
 3   Low           8004 non-null   float64                  
 4   Close         8004 non-null   float64                  
 5   Volume        8004 non-null   int64                    
 6   Dividends     8004 non-null   float64                  
 7   Stock Splits  8004 non-null   float64                  
dtypes: datetime64[ns, UTC-05:00](1), float64(6), int64(1)
memory usage: 500.4 KB
